In [4]:
import numpy as np
from bayesian_optimization import BayesianOptimization, BayesianOptimization1dDemo

In [10]:
def f(x):
    return -(5 * x - 2)**2 * np.sin(12 * x - 4)

bo = BayesianOptimization1dDemo(f, np.array([[0.0, 1.0]]))
X, y = bo.fit()

In [11]:
from IPython.display import HTML
HTML('<img src="output.gif">')